In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , mean_absolute_error
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
import joblib

2025-07-01 10:36:20.982070: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-01 10:36:20.990037: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-01 10:36:21.011658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751358981.042550  225526 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751358981.049764  225526 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751358981.073411  225526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

### Load model :

In [2]:
model=load_model('/home/fayari/Stage/modeling/deep_model_v2.h5', compile=False)

2025-06-30 10:36:23.464401: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Load Data :

In [3]:
duplicated_data=pd.read_csv('/home/fayari/Stage/data_exploration_2/noisy_duplicated_data.csv')
df_no_outliers=pd.read_csv('/home/fayari/Stage/data_exploration/no_outliers.csv')
df_no_outliers_SDB = pd.read_csv('/home/fayari/Stage/data_exploration_2/no_outliers.csv')
df_no_outliers_2_SDB = pd.read_csv('/home/fayari/Stage/data_exploration_2/no_outliers_2.csv')
augmented_data=pd.read_csv('/home/fayari/Stage/data_augmentation/add_training.csv')
#df_no_outliers.drop(columns=['BOA_RT'],inplace=True)
#new = pd.read_csv('/home/fayari/Stage/modeling_BOA/add_training_v2.csv')
data=pd.concat([duplicated_data,df_no_outliers,df_no_outliers_SDB,df_no_outliers_2_SDB,augmented_data], ignore_index=True)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data['BOA_fraction']=data['BOA_RT'] / 1000
data.drop(columns=['alpha','BOA_RT'],inplace=True)

In [4]:
scaler=joblib.load('/home/fayari/Stage/modeling/scaler_SS_BOA_DART.pkl')
scaler_y=joblib.load('/home/fayari/Stage/modeling/scaler_y_SS_BOA_DART.pkl')

In [5]:
cols_to_scale = data.drop(columns=['BOA_fraction']).columns

In [6]:
X_scaled = scaler.transform(data[cols_to_scale])
y_scaled=scaler_y.transform(data['BOA_fraction'].values.reshape(-1, 1))
BOA=data['BOA_fraction']

In [7]:
X_train,X_val,y_train,y_val,BOA_train,BOA_val=train_test_split(X_scaled,y_scaled,BOA,test_size=0.2,random_state=1042)

In [8]:
y_pred=model.predict(X_val)

26203/26203 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step


In [9]:
y_pred

array([[ 0.12375316],
       [ 0.49810976],
       [ 0.9530338 ],
       ...,
       [-0.11332791],
       [ 0.20404471],
       [ 0.24837866]], dtype=float32)

In [10]:
X_scaled

array([[ 0.22641612,  0.18255773,  0.27001448, ...,  1.36668976,
        -0.54567663, -0.47147194],
       [-0.12076683,  0.40664931,  0.2401108 , ...,  0.40066368,
        -0.4389886 , -0.37316836],
       [ 0.17524403,  0.24193393,  0.26541392, ..., -1.47121589,
         1.12538622,  0.56828789],
       ...,
       [-1.50241327,  0.47792528,  0.15730064, ...,  1.37056136,
        -0.53223094, -0.43237924],
       [ 0.54210856,  0.29375892,  0.30911929, ..., -0.12277564,
        -0.31843119, -0.22486828],
       [-0.90626657,  0.47681299,  0.18720431, ..., -1.30775753,
         0.58327199,  0.30666087]])

In [11]:
X_train = pd.DataFrame(X_train, columns=data[cols_to_scale].columns)
X_val = pd.DataFrame(X_val, columns=data[cols_to_scale].columns)
X_train

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a
0,0.353953,0.264667,0.283816,0.414692,-0.339098,-0.080134,-0.001448,-1.493189,-0.662371,-0.605936,0.106222,1.300565,-0.662371,-0.662371,1.304263,1.301594,-0.519276,-0.401373
1,-0.506526,0.452311,0.210207,0.454404,0.323859,0.476440,0.590898,0.980637,-0.493514,0.625269,-0.367488,-1.035347,-0.493514,-0.493514,-1.041293,-1.037128,0.163859,0.116563
2,0.797969,-0.085427,0.283816,-0.356965,-0.651614,-1.136093,-1.164179,-1.106653,-0.845089,-0.531502,0.670002,1.138866,-0.845089,-0.845089,1.141544,1.139621,-0.502541,-0.376283
3,0.790884,0.404903,0.224633,-0.275700,-0.646768,-0.991697,-1.021665,-1.725110,-0.115476,0.350947,0.711006,1.352518,-0.115476,-0.115476,1.356546,1.353635,-0.533859,-0.440300
4,-1.502413,0.477925,0.157301,0.466141,1.209870,0.832674,0.968372,0.864676,0.336909,-0.666119,-0.130011,-0.875279,0.336909,0.336909,-0.881654,-0.877115,-0.045002,-0.168410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353853,-0.413143,-0.714113,-2.536682,-1.578721,0.247820,0.976541,0.324313,0.305196,6.162575,1.397261,-0.415296,-0.146344,6.162575,6.162575,-0.150900,-0.147569,-0.513597,-0.564907
3353854,0.076836,0.319631,0.258513,0.444787,-0.134402,0.176427,0.274013,0.284873,-0.173442,-1.500406,-0.638211,-0.121583,-0.173442,-0.173442,-0.126024,-0.122776,-0.341525,-0.304253
3353855,-0.794474,0.464378,0.192361,0.458077,0.565356,0.592169,0.713347,0.091606,-0.780823,-0.418842,-0.281996,0.106529,-0.780823,-0.780823,0.103200,0.105650,-0.370441,-0.269993
3353856,0.799544,0.300258,0.286116,-0.383952,-0.652691,-1.152223,-1.181301,-1.493189,-0.399004,1.544985,0.657102,1.300565,-0.399004,-0.399004,1.304263,1.301594,-0.524611,-0.419915


In [12]:
y_pred_val_real=scaler_y.inverse_transform(y_pred)

In [13]:
y_pred_val_real

array([[0.7690971 ],
       [0.860682  ],
       [0.9719775 ],
       ...,
       [0.7110961 ],
       [0.7887401 ],
       [0.79958624]], dtype=float32)

In [14]:
y_val_real=scaler_y.inverse_transform(y_val)

In [15]:
X_val_inverse = scaler.inverse_transform(X_val)

# Recréer un DataFrame avec les noms de colonnes d'origine
X_val_real = pd.DataFrame(X_val_inverse, columns=data[cols_to_scale].columns)
X_val_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a
0,0.965735,0.979888,0.985700,0.938760,0.034865,0.235193,0.220790,75.000000,0.00000,0.150000,0.950950,0.258819,707.888889,3185.500000,0.261337,0.259383,3.826474,3.855296
1,0.913600,0.963253,0.984000,0.949213,0.090362,0.317592,0.301462,54.000000,1331.00000,0.092989,0.879863,0.587785,708.036778,3186.165500,0.588569,0.587960,1.465463,0.874237
2,0.982900,0.997105,0.983800,0.910409,0.017248,0.182302,0.165969,48.000000,992.00000,0.213061,0.947507,0.669131,707.999111,3185.996000,0.669712,0.669260,1.337343,0.909903
3,0.996600,0.945571,0.977354,0.844324,0.003406,0.147143,0.124236,82.000000,172.00000,0.309940,0.936597,0.139173,707.908000,3185.586000,0.143977,0.140270,6.814072,6.541626
4,0.985800,0.999600,0.983700,0.903454,0.014302,0.170223,0.153788,86.000000,2609.00000,0.638102,0.946475,0.069756,708.178778,3186.804500,0.078675,0.071927,9.511873,3.772017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838460,0.895400,0.972267,0.983500,0.950343,0.110485,0.335053,0.318416,13.000000,514.00000,0.084474,0.877362,0.974370,707.946000,3185.757000,0.974407,0.974378,0.969297,0.793704
838461,0.999181,0.704870,0.837217,0.695953,0.000819,0.584355,0.406683,24.653494,523.67282,0.730534,0.937760,0.908847,707.947075,3185.761836,0.908982,0.908877,1.037946,0.846800
838462,0.835901,0.864755,0.827425,0.383499,0.179245,0.307278,0.117841,17.138622,873.83824,0.364132,0.924310,0.955595,707.985982,3185.936919,0.955659,0.955609,0.949577,0.676034
838463,0.956700,0.983800,0.985700,0.943378,0.044265,0.254376,0.239973,78.000000,1061.00000,0.404736,0.938833,0.207912,708.006778,3186.030500,0.211108,0.208631,4.210146,2.819860


In [16]:
X_val_real['BOA_fraction']=y_val_real
X_val_real['BOA_fraction_pred']=y_pred_val_real

In [17]:
X_val_real['BOA_RT']=X_val_real['BOA_fraction'] * 1000
X_val_real['BOA_pred']=X_val_real['BOA_fraction_pred'] * 1000
X_val_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,...,mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred
0,0.965735,0.979888,0.985700,0.938760,0.034865,0.235193,0.220790,75.000000,0.00000,0.150000,...,707.888889,3185.500000,0.261337,0.259383,3.826474,3.855296,0.767686,0.769097,767.686042,769.097107
1,0.913600,0.963253,0.984000,0.949213,0.090362,0.317592,0.301462,54.000000,1331.00000,0.092989,...,708.036778,3186.165500,0.588569,0.587960,1.465463,0.874237,0.861060,0.860682,861.060447,860.682007
2,0.982900,0.997105,0.983800,0.910409,0.017248,0.182302,0.165969,48.000000,992.00000,0.213061,...,707.999111,3185.996000,0.669712,0.669260,1.337343,0.909903,0.972654,0.971977,972.653514,971.977478
3,0.996600,0.945571,0.977354,0.844324,0.003406,0.147143,0.124236,82.000000,172.00000,0.309940,...,707.908000,3185.586000,0.143977,0.140270,6.814072,6.541626,0.548045,0.554754,548.045045,554.753662
4,0.985800,0.999600,0.983700,0.903454,0.014302,0.170223,0.153788,86.000000,2609.00000,0.638102,...,708.178778,3186.804500,0.078675,0.071927,9.511873,3.772017,0.834227,0.840092,834.226789,840.091980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838460,0.895400,0.972267,0.983500,0.950343,0.110485,0.335053,0.318416,13.000000,514.00000,0.084474,...,707.946000,3185.757000,0.974407,0.974378,0.969297,0.793704,0.909084,0.909086,909.084198,909.086426
838461,0.999181,0.704870,0.837217,0.695953,0.000819,0.584355,0.406683,24.653494,523.67282,0.730534,...,707.947075,3185.761836,0.908982,0.908877,1.037946,0.846800,0.598902,0.600223,598.901989,600.223450
838462,0.835901,0.864755,0.827425,0.383499,0.179245,0.307278,0.117841,17.138622,873.83824,0.364132,...,707.985982,3185.936919,0.955659,0.955609,0.949577,0.676034,0.709756,0.711096,709.755660,711.096130
838463,0.956700,0.983800,0.985700,0.943378,0.044265,0.254376,0.239973,78.000000,1061.00000,0.404736,...,708.006778,3186.030500,0.211108,0.208631,4.210146,2.819860,0.786490,0.788740,786.489733,788.740112


### Validation Mean Absolute Error :

In [18]:
MAE=mean_absolute_error(X_val_real['BOA_RT'],X_val_real['BOA_pred'])
MAE

1.8502506002271046

### Same Thing for training :

In [19]:
y_pred=model.predict(X_train)
y_pred_train_real=scaler_y.inverse_transform(y_pred)
y_train_real=scaler_y.inverse_transform(y_train)
 

104809/104809 ━━━━━━━━━━━━━━━━━━━━ 157s 1ms/step


In [20]:
X_train_inverse = scaler.inverse_transform(X_train)

# Recréer un DataFrame avec les noms de colonnes d'origine
X_train_real = pd.DataFrame(X_train_inverse, columns=data[cols_to_scale].columns)
X_train_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a
0,0.940200,0.959163,0.984900,0.946064,0.061663,0.282098,0.266883,12.000000,162.000000,0.278167,0.910085,0.978148,707.906889,3185.58100,0.978179,0.978155,1.004071,0.942789
1,0.830900,0.992632,0.981700,0.951314,0.185246,0.379359,0.360889,76.000000,296.000000,0.639305,0.887843,0.241922,707.921778,3185.64800,0.244636,0.242531,3.955449,3.555963
2,0.996600,0.896720,0.984900,0.844057,0.003406,0.097569,0.082354,22.000000,17.000000,0.300000,0.936556,0.927184,707.890778,3185.50850,0.927291,0.927208,1.076376,1.069379
3,0.995700,0.984176,0.982327,0.854800,0.004309,0.122802,0.104971,6.000000,596.000000,0.558841,0.938481,0.994522,707.955111,3185.79800,0.994530,0.994524,0.941071,0.746389
4,0.704400,0.997200,0.979400,0.952865,0.350409,0.441611,0.420795,73.000000,955.000000,0.260514,0.898993,0.292372,707.995000,3185.97750,0.294561,0.292862,3.053100,2.118175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353853,0.842762,0.784586,0.862285,0.682551,0.171071,0.466751,0.318582,58.525755,5578.080638,0.865747,0.885598,0.522115,708.508676,3188.28904,0.523095,0.522334,1.028607,0.117703
3353854,0.905000,0.968967,0.983800,0.950043,0.099820,0.326932,0.310599,58.000000,550.000000,0.015800,0.875132,0.529919,707.950000,3185.77500,0.530875,0.530132,1.772016,1.432798
3353855,0.794324,0.994784,0.980924,0.951799,0.230264,0.399582,0.380322,53.000000,68.000000,0.333046,0.891857,0.601815,707.896444,3185.53400,0.602562,0.601981,1.647090,1.605650
3353856,0.996800,0.965511,0.985000,0.840490,0.003205,0.094750,0.079637,12.000000,371.000000,0.909077,0.935950,0.978148,707.930111,3185.68550,0.978179,0.978155,0.981023,0.849241


In [21]:
X_train_real['BOA_fraction']=y_train_real
X_train_real['BOA_fraction_pred']=y_pred_train_real

In [22]:
X_train_real['BOA_RT']=X_train_real['BOA_fraction'] * 1000
X_train_real['BOA_pred']=X_train_real['BOA_fraction_pred'] * 1000
X_train_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scene,...,mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred
0,0.940200,0.959163,0.984900,0.946064,0.061663,0.282098,0.266883,12.000000,162.000000,0.278167,...,707.906889,3185.58100,0.978179,0.978155,1.004071,0.942789,0.929803,0.929903,929.803116,929.903015
1,0.830900,0.992632,0.981700,0.951314,0.185246,0.379359,0.360889,76.000000,296.000000,0.639305,...,707.921778,3185.64800,0.244636,0.242531,3.955449,3.555963,0.663949,0.663775,663.948897,663.774719
2,0.996600,0.896720,0.984900,0.844057,0.003406,0.097569,0.082354,22.000000,17.000000,0.300000,...,707.890778,3185.50850,0.927291,0.927208,1.076376,1.069379,0.874454,0.874009,874.453646,874.009277
3,0.995700,0.984176,0.982327,0.854800,0.004309,0.122802,0.104971,6.000000,596.000000,0.558841,...,707.955111,3185.79800,0.994530,0.994524,0.941071,0.746389,0.977893,0.976121,977.892849,976.121399
4,0.704400,0.997200,0.979400,0.952865,0.350409,0.441611,0.420795,73.000000,955.000000,0.260514,...,707.995000,3185.97750,0.294561,0.292862,3.053100,2.118175,0.621174,0.620669,621.174436,620.669128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353853,0.842762,0.784586,0.862285,0.682551,0.171071,0.466751,0.318582,58.525755,5578.080638,0.865747,...,708.508676,3188.28904,0.523095,0.522334,1.028607,0.117703,0.736061,0.739680,736.061333,739.679688
3353854,0.905000,0.968967,0.983800,0.950043,0.099820,0.326932,0.310599,58.000000,550.000000,0.015800,...,707.950000,3185.77500,0.530875,0.530132,1.772016,1.432798,0.819204,0.819222,819.204095,819.222229
3353855,0.794324,0.994784,0.980924,0.951799,0.230264,0.399582,0.380322,53.000000,68.000000,0.333046,...,707.896444,3185.53400,0.602562,0.601981,1.647090,1.605650,0.827250,0.826563,827.249904,826.563416
3353856,0.996800,0.965511,0.985000,0.840490,0.003205,0.094750,0.079637,12.000000,371.000000,0.909077,...,707.930111,3185.68550,0.978179,0.978155,0.981023,0.849241,0.962139,0.961329,962.138651,961.328674


In [23]:
MAE=mean_absolute_error(X_train_real['BOA_RT'],X_train_real['BOA_pred'])
MAE

1.8357421057504868